- https://github.com/JuliaReach/ReachabilityBenchmarks/pull/205

In [4]:
using ReachabilityModels, SparseArrays, MAT

M_pde = vec(matread("out_pde.mat")["M"])
M_iss = vec(matread("out_iss.mat")["M"])

y = [false, false, true, false, true, false]
models = ["motor", "building", "pde", "heat", "iss", "beam"]
dirs = [5, 25, M_pde, 133, M_iss, 89]
conditions = [x -> isdisjoint(x, Interval(0.45, 0.6)),       # x5 < 0.45 || x5 > 0.6
              x -> isdisjoint(x, HalfSpace([-1.], -6e-3)),   # x25 < 6e-3
              x -> isdisjoint(x, HalfSpace([-1.], -12.)),    # y1 < 12
              x -> isdisjoint(x, HalfSpace([-1.], -0.1)),    # x133 < 0.1
              x -> x ⊆ (Interval(-7., 7.) * 1e-4),           # -7e-4 <= y1 <= 7e-4
              x -> isdisjoint(x, HalfSpace([-1.], -2100.))]  # x89 < 2100
sols = []
ints = []

for i=1:length(models)
    println(models[i])
    m = fetch_model(models[i])
    meta = fetch_meta(models[i])
    dim = meta["info"]["dim"]
   
    x = y[i] ? dirs[i] : SingleEntryVector(dirs[i], dim, 1.0)

    @time sol = solve(m, T=20, alg=LGG09(δ=0.001, template=[x, -x]))
    push!(sols, sol)

    if !y[i]
        int = Interval(-ρ(sparsevec([dirs[i]], [-1.0], dim), sols[i]), ρ(sparsevec([dirs[i]], [1.0], dim), sols[i]))
    else
        int = Interval(-ρ(-x, sols[i]), ρ(x, sols[i]))
    end
    push!(ints, int)
    if conditions[i](int)
        println("verified")
    else
        println("not verified")
    end
end

motor
  0.880995 seconds (1.93 M allocations: 109.413 MiB, 3.27% gc time)
verified


building
  0.421222 seconds (954.86 k allocations: 76.944 MiB, 5.42% gc time)


verified
pde
  0.250911 seconds (123.63 k allocations: 57.430 MiB, 9.98% gc time)
verified


heat
  0.938629 seconds (959.40 k allocations: 211.311 MiB, 4.00% gc time)


verified
iss
  1.181445 seconds (128.92 k allocations: 216.154 MiB, 1.35% gc time)
not verified


beam
  2.879968 seconds (138.41 k allocations: 431.019 MiB, 2.79% gc time)


not verified


## Heat

In [29]:
m = fetch_model("heat");

In [30]:
x = SingleEntryVector(133, statedim(m), 1.0);

In [28]:
@time sol = solve(m, T=20, alg=LGG09(δ=0.001, template=[x]));

  0.471040 seconds (213.34 k allocations: 135.985 MiB, 1.82% gc time)


In [31]:
@time sol = solve(m, T=20, alg=LGG09(δ=0.001, template=[x, -x]));

  0.629519 seconds (126.67 k allocations: 166.682 MiB, 2.65% gc time)


In [44]:
using ReachabilityAnalysis: support_functions
const sf = support_functions

Interval(-sf(sol[end], 2), sf(sol[end], 1))

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.056793, 0.056862])

In [45]:
Interval(-ρ(-x, sol[end]), ρ(x, sol[end]))

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.056793, 0.056862])

In [26]:
@time sol = solve(m, T=20, alg=LGG09(δ=0.001, template=BoxDirections(200)));

 98.378818 seconds (25.25 M allocations: 15.115 GiB, 2.98% gc time)


In [27]:
sol[100]

TemplateReachSet{Float64,SingleEntryVector{Float64},BoxDirections{Float64,SingleEntryVector{Float64}},SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}}(BoxDirections{Float64,SingleEntryVector{Float64}}(200), [0.0038693861679344682, 0.007598026817837309, 0.011053707324852856, 0.01412055108158657, 0.01670545801894523, 0.01874267703810714, 0.02019621124951963, 0.021059975094797918, 0.021355839149301514, 0.021129886361332936  …  -0.00042341234845319053, -0.0004507910575348701, -0.0004647800128299329, -0.00046289314512956654, -0.00044343128305389146, -0.00040570544437817143, -0.0003501859593940479, -0.0002785545675718009, -0.00019364651666899246, -9.928206551699323e-5], [0.0989999, 0.100001])

In [18]:
numrsets(sol)

20000

In [19]:
using Plots

In [23]:
#plot(sol, vars=(0, 133), xlab="t", ylab="x133")